In [2]:
import requests
import pandas as pd

def get_all_rickmorty_characters():
    url = "https://rickandmortyapi.com/api/character"
    characters = []

    while url:
        response = requests.get(url).json()
        for c in response["results"]:
            characters.append({
                "id": c["id"],
                "name": c["name"],
                "status": c["status"],
                "species": c["species"],
                "type": c["type"],
                "gender": c["gender"],
                "origin": c["origin"]["name"],
                "location": c["location"]["name"],
                "image": c["image"],
                "episode_count": len(c["episode"])
            })
        url = response["info"]["next"]  # pagination

    return pd.DataFrame(characters)

characters_df = get_all_rickmorty_characters()
characters_df.head()


,id,name,status,species,type,gender,origin,location,image,episode_count
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,51
1,2,Morty Smith,Alive,Human,,Male,unknown,Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,51
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,42
3,4,Beth Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,42
4,5,Jerry Smith,Alive,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,39


In [3]:
def get_all_rickmorty_episodes():
    url = "https://rickandmortyapi.com/api/episode"
    episodes = []

    while url:
        response = requests.get(url).json()
        for e in response["results"]:
            episodes.append({
                "id": e["id"],
                "name": e["name"],
                "air_date": e["air_date"],
                "episode_code": e["episode"],
                "character_count": len(e["characters"])
            })
        url = response["info"]["next"]

    return pd.DataFrame(episodes)

episodes_df = get_all_rickmorty_episodes()
episodes_df.head()


,id,name,air_date,episode_code,character_count
0,1,Pilot,"December 2, 2013",S01E01,19
1,2,Lawnmower Dog,"December 9, 2013",S01E02,19
2,3,Anatomy Park,"December 16, 2013",S01E03,24
3,4,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,13
4,5,Meeseeks and Destroy,"January 20, 2014",S01E05,20


In [4]:
def get_all_rickmorty_locations():
    url = "https://rickandmortyapi.com/api/location"
    locations = []

    while url:
        response = requests.get(url).json()
        for l in response["results"]:
            locations.append({
                "id": l["id"],
                "name": l["name"],
                "type": l["type"],
                "dimension": l["dimension"],
                "resident_count": len(l["residents"])
            })
        url = response["info"]["next"]

    return pd.DataFrame(locations)

locations_df = get_all_rickmorty_locations()
locations_df.head()


,id,name,type,dimension,resident_count
0,1,Earth (C-137),Planet,Dimension C-137,27
1,2,Abadango,Cluster,unknown,1
2,3,Citadel of Ricks,Space station,unknown,101
3,4,Worldender's lair,Planet,unknown,9
4,5,Anatomy Park,Microverse,Dimension C-137,11


In [6]:
with pd.ExcelWriter("13.Rick and Morty API Data.xlsx") as writer:
    characters_df.to_excel(writer, sheet_name="Characters", index=False)
    episodes_df.to_excel(writer, sheet_name="Episodes", index=False)
    locations_df.to_excel(writer, sheet_name="Locations", index=False)


In [7]:
import sqlite3

conn = sqlite3.connect(":memory:")  # temporary SQL database in RAM

characters_df.to_sql("characters", conn, index=False, if_exists="replace")
episodes_df.to_sql("episodes", conn, index=False, if_exists="replace")
locations_df.to_sql("locations", conn, index=False, if_exists="replace")


126

In [8]:
query = """
SELECT 
    c.id AS character_id,
    c.name AS character_name,
    c.status,
    c.species,
    c.type,
    c.gender,
    c.episode_count,

    -- origin info
    o.id AS origin_id,
    o.type AS origin_type,
    o.dimension AS origin_dimension,

    -- current location info
    l.id AS location_id,
    l.type AS location_type,
    l.dimension AS location_dimension

FROM characters c
LEFT JOIN locations o ON c.origin = o.name
LEFT JOIN locations l ON c.location = l.name
"""


In [9]:
merged_df = pd.read_sql_query(query, conn)
merged_df.head()


,character_id,character_name,status,species,type,gender,episode_count,origin_id,origin_type,origin_dimension,location_id,location_type,location_dimension
0,1,Rick Sanchez,Alive,Human,,Male,51,1.0,Planet,Dimension C-137,3.0,Space station,unknown
1,2,Morty Smith,Alive,Human,,Male,51,NaN,None,None,3.0,Space station,unknown
2,3,Summer Smith,Alive,Human,,Female,42,20.0,Planet,Replacement Dimension,20.0,Planet,Replacement Dimension
3,4,Beth Smith,Alive,Human,,Female,42,20.0,Planet,Replacement Dimension,20.0,Planet,Replacement Dimension
4,5,Jerry Smith,Alive,Human,,Male,39,20.0,Planet,Replacement Dimension,20.0,Planet,Replacement Dimension


In [10]:
merged_df

,character_id,character_name,status,species,type,gender,episode_count,origin_id,origin_type,origin_dimension,location_id,location_type,location_dimension
0,1,Rick Sanchez,Alive,Human,,Male,51,1.0,Planet,Dimension C-137,3.0,Space station,unknown
1,2,Morty Smith,Alive,Human,,Male,51,NaN,None,None,3.0,Space station,unknown
2,3,Summer Smith,Alive,Human,,Female,42,20.0,Planet,Replacement Dimension,20.0,Planet,Replacement Dimension
3,4,Beth Smith,Alive,Human,,Female,42,20.0,Planet,Replacement Dimension,20.0,Planet,Replacement Dimension
4,5,Jerry Smith,Alive,Human,,Male,39,20.0,Planet,Replacement Dimension,20.0,Planet,Replacement Dimension
...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,822,Young Jerry,unknown,Human,,Male,1,30.0,Planet,unknown,30.0,Planet,unknown
822,823,Young Beth,unknown,Human,,Female,1,30.0,Planet,unknown,30.0,Planet,unknown
823,824,Young Beth,unknown,Human,,Female,1,30.0,Planet,unknown,30.0,Planet,unknown
824,825,Young Jerry,unknown,Human,,Male,1,30.0,Planet,unknown,30.0,Planet,unknown


In [11]:
merged = (
    characters_df
    .merge(episodes_df, on="id")
    .merge(locations_df, on="id")
)


In [12]:
merged

,id,name_x,status,species,type_x,gender,origin,location,image,episode_count,name_y,air_date,episode_code,character_count,name,type_y,dimension,resident_count
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,51,Pilot,"December 2, 2013",S01E01,19,Earth (C-137),Planet,Dimension C-137,27
1,2,Morty Smith,Alive,Human,,Male,unknown,Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,51,Lawnmower Dog,"December 9, 2013",S01E02,19,Abadango,Cluster,unknown,1
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,42,Anatomy Park,"December 16, 2013",S01E03,24,Citadel of Ricks,Space station,unknown,101
3,4,Beth Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,42,M. Night Shaym-Aliens!,"January 13, 2014",S01E04,13,Worldender's lair,Planet,unknown,9
4,5,Jerry Smith,Alive,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,39,Meeseeks and Destroy,"January 20, 2014",S01E05,20,Anatomy Park,Microverse,Dimension C-137,11
5,6,Abadango Cluster Princess,Alive,Alien,,Female,Abadango,Abadango,https://rickandmortyapi.com/api/character/avat...,1,Rick Potion #9,"January 27, 2014",S01E06,24,Interdimensional Cable,TV,unknown,62
6,7,Abradolf Lincler,unknown,Human,Genetic experiment,Male,Earth (Replacement Dimension),Testicle Monster Dimension,https://rickandmortyapi.com/api/character/avat...,2,Raising Gazorpazorp,"March 10, 2014",S01E07,14,Immortality Field Resort,Resort,unknown,3
7,8,Adjudicator Rick,Dead,Human,,Male,unknown,Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,1,Rixty Minutes,"March 17, 2014",S01E08,57,Post-Apocalyptic Earth,Planet,Post-Apocalyptic Dimension,10
8,9,Agency Director,Dead,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,1,Something Ricked This Way Comes,"March 24, 2014",S01E09,17,Purge Planet,Planet,Replacement Dimension,4
9,10,Alan Rails,Dead,Human,Superhuman (Ghost trains summoner),Male,unknown,Worldender's lair,https://rickandmortyapi.com/api/character/avat...,1,Close Rick-counters of the Rick Kind,"April 7, 2014",S01E10,60,Venzenulon 7,Planet,unknown,1


In [13]:
merged1 = (
    characters_df
    .merge(locations_df.add_prefix("origin_"), left_on="origin", right_on="origin_name", how="left")
    .merge(locations_df.add_prefix("loc_"), left_on="location", right_on="loc_name", how="left")
)


In [14]:
merged1

,id,name,status,species,type,gender,origin,location,image,episode_count,origin_id,origin_name,origin_type,origin_dimension,origin_resident_count,loc_id,loc_name,loc_type,loc_dimension,loc_resident_count
0,1,Rick Sanchez,Alive,Human,,Male,Earth (C-137),Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,51,1.0,Earth (C-137),Planet,Dimension C-137,27.0,3.0,Citadel of Ricks,Space station,unknown,101.0
1,2,Morty Smith,Alive,Human,,Male,unknown,Citadel of Ricks,https://rickandmortyapi.com/api/character/avat...,51,NaN,NaN,NaN,NaN,NaN,3.0,Citadel of Ricks,Space station,unknown,101.0
2,3,Summer Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,42,20.0,Earth (Replacement Dimension),Planet,Replacement Dimension,230.0,20.0,Earth (Replacement Dimension),Planet,Replacement Dimension,230.0
3,4,Beth Smith,Alive,Human,,Female,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,42,20.0,Earth (Replacement Dimension),Planet,Replacement Dimension,230.0,20.0,Earth (Replacement Dimension),Planet,Replacement Dimension,230.0
4,5,Jerry Smith,Alive,Human,,Male,Earth (Replacement Dimension),Earth (Replacement Dimension),https://rickandmortyapi.com/api/character/avat...,39,20.0,Earth (Replacement Dimension),Planet,Replacement Dimension,230.0,20.0,Earth (Replacement Dimension),Planet,Replacement Dimension,230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
821,822,Young Jerry,unknown,Human,,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,1,30.0,Earth (Unknown dimension),Planet,unknown,4.0,30.0,Earth (Unknown dimension),Planet,unknown,4.0
822,823,Young Beth,unknown,Human,,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,1,30.0,Earth (Unknown dimension),Planet,unknown,4.0,30.0,Earth (Unknown dimension),Planet,unknown,4.0
823,824,Young Beth,unknown,Human,,Female,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,1,30.0,Earth (Unknown dimension),Planet,unknown,4.0,30.0,Earth (Unknown dimension),Planet,unknown,4.0
824,825,Young Jerry,unknown,Human,,Male,Earth (Unknown dimension),Earth (Unknown dimension),https://rickandmortyapi.com/api/character/avat...,1,30.0,Earth (Unknown dimension),Planet,unknown,4.0,30.0,Earth (Unknown dimension),Planet,unknown,4.0


In [16]:
with pd.ExcelWriter("13.Rick and Morty API Data.xlsx", mode="a", engine="openpyxl") as writer:
    merged_df.to_excel(writer, sheet_name="MergedData", index=False)
